In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode 

In [ ]:
df_movies = pd.read_csv("Movies_EDA.csv", sep=",")

In [ ]:
df_movies.info()

Solo tomaremos las columnas que necesitemos para el sistema de recomendación.

In [ ]:
df_movies = df_movies[["title", "overview", "release_year", "genres_names", "runtime"]].copy()

In [ ]:
df_movies.info()

Eliminamos los duplicados de la columna title.

In [ ]:
df_movies.drop_duplicates(subset=['title'],inplace=True)
df_movies=df_movies.reset_index(drop=True)
df_movies.shape

In [ ]:
print(df_movies.head)

        ► Sistema de recomendación.

Se realizará un sistema de recomendación de películas basados en el resumen y el género de la película de entrada. 

El resultado incluirá el título de la película, el año de estreno y la duración de la película recomendada.


Lo que se realizó en la función fue:

*Normalizamos el título ingresado por el usuario para que no distinga entre mayúsculas y minúsculas, y no tenga en cuenta los acentos.*

*Se obtiene la película de entrada por título en la base de datos.*
 
*Calculamos la similitud entre la película de entrada y las demás películas en función del resumen.*
    
*Se filtró las películas similares por género.*
    
*Se ordenó las películas similares según la similitud.*
    
*Conversión de los datos de duración de las recomendaciones de tipo float a int.*
    
*Retorno de las películas recomendadas, incluyendo el título, el año de estreno y la duración.*

In [90]:
def normalize_text(text):
    # Convertimos el texto a minúsculas y removemos acentos
        if isinstance(text, str):
            text = unidecode(text.lower())
        else:
            text = text.apply(lambda x: unidecode(x.lower()))
        return text

In [91]:
def recommend_movies(title):
    # Normalizamos el título sin diferenciar acentos
    title = normalize_text(title)
    
    # Obtenemos la película de entrada por título
    movie = df_movies[normalize_text(df_movies['title'].apply(normalize_text)) == title]
    
    if len(movie) == 0:
        return "No se encontró la película en la base de datos."
    
    # Obtenemos el índice de la película de entrada
    index = movie.index[0]
    
    # Obtenemos el resumen y género de la película de entrada
    overview = df_movies.loc[index, 'overview']
    genre = df_movies.loc[index, 'genres_names']
    
    # Calculamos la similitud basada en el resumen
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['overview'])
    overview_similarities = cosine_similarity(tfidf_matrix[index], tfidf_matrix).flatten()
    
    # Filtramos películas similares por género
    similar_movies_indices = overview_similarities.argsort()[::-1]
    similar_movies = df_movies.loc[similar_movies_indices]
    similar_movies = similar_movies[similar_movies['genres_names'].str.contains(genre)].head(6)
    
    # Eliminamos la película de entrada de las recomendaciones
    similar_movies = similar_movies[similar_movies.index != index]
    
    # Convertimos el tiempo a enteros
    similar_movies['runtime'] = similar_movies['runtime'].astype(int)
    
    return similar_movies[['title', 'release_year', 'runtime']]


In [92]:
title = input("Ingrese el título de la película: ")
recommendations = recommend_movies(title)

if isinstance(recommendations, str):
    print(recommendations)
else:
    print(recommendations)

                         title  release_year  runtime
9107        Fear and Trembling          2003      107
24142        The Walking Stick          1970       96
28950                   Quints          2000       83
16387  The High Cost of Living          2010       92
38339            Goin' to Town          1935       74
